In [12]:
import cv2
from ultralytics import YOLO
import numpy as np

In [11]:
#capture video camera
cap_video=cv2.VideoCapture(0)
width=int(cap_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_count=int(cap_video.get(cv2.CAP_PROP_FRAME_COUNT))
#DIVX for windows 
writer= cv2.VideoWriter("my_video.mp4",cv2.VideoWriter_fourcc(*'DIVX'),20,(width,height))
while True:
    ret, frame= cap_video.read()
    #gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    writer.write(frame)
    cv2.imshow("my image",frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap_video.release()
writer.release()
cv2.destroyAllWindows()

In [20]:


# Load YOLOv8 model
model = YOLO("yolov8n.pt")  

# Initialize video capture
video_path = "people1.avi"  
cap = cv2.VideoCapture(video_path)

boundary_line= 540
line_color=(0,255,0)
line_thickenss=4
#person in and out
person_in=0
person_out=0

#centroid and cross people
centroid={}
cross_people=set()

# Check if the video was successfully loaded
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished processing video.")
        break

    # Run YOLO model for detection
    results = model(frame)

    # Extract bounding boxes and class IDs
    detections = results[0].boxes.data.cpu().numpy()  # YOLOv8 outputs data as a tensor
    count = 0
    new_centroids = {}
    current_id = 0
    
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        class_id = int(class_id)
        
        # Class ID for 'person' in COCO dataset is 0
        if class_id == 0:
            count += 1
            #draw centroid
            centroid_x=int((x1+x2)/2)
            centroid_y=int((y1+y2)/2)
            cv2.circle(frame,(centroid_x,centroid_y),5,(255,0,0),-1)
            new_centroids[current_id]=(centroid_x,centroid_y)
            current_id += 1
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f"Person {count}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
    # Display count on frame
    cv2.putText(frame, f"People Count: {count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    # Show frame
    cv2.imshow("YOLOv8 People Counter", small_frame)

    # Exit with 'q' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 384x640 14 persons, 2 cars, 1 skis, 166.2ms
Speed: 19.4ms preprocess, 166.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 cars, 1 skis, 175.2ms
Speed: 28.7ms preprocess, 175.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 2 cars, 1 backpack, 1 skis, 161.9ms
Speed: 2.9ms preprocess, 161.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 cars, 1 backpack, 1 handbag, 150.5ms
Speed: 8.0ms preprocess, 150.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 cars, 1 backpack, 1 handbag, 1 skis, 166.4ms
Speed: 13.9ms preprocess, 166.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 2 cars, 1 backpack, 1 handbag, 1 skis, 159.6ms
Speed: 11.8ms preprocess, 159.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 2 cars, 1 backpack, 1 h